# Image classifier

## Info
ArdavanBidgoli <br>
CMU School of Architecture <br>
Robotic Plastering Project <br>
Feedback-loop image classifier <br>
Tested with/for:
    Tensorflow 0.12.1
    OpenCV 3.2.0-dev

this code has been inspired by:
https://codelabs.developers.google.com/codelabs/tensorflow-for-poets/index.html?index=../../index#4


## Imports

In [ ]:
# Import Tensorflow
import tensorflow as tf

# Import libraries for:
# System read and write, Checking object types, Time keeping
import sys
import time
import os
from os import listdir
from os.path import isfile, join, exists
from shutil import copyfile

# import json for json formatting
import json

# import openCV
import cv2

# import numpy
import numpy as np

# import matPlotLib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches


print (os.getcwd())

## General Setup

In [ ]:
# General Setup
######################################################################
# print in-progress report
printSwitch = True
# Sets the naming standard
sampleFolder = "./tests"
# Set log file info
logFolder = "./log"
logFileName = "log"
# keep track of time
start_time = time.time()

# Error messages:
nameFinderError= "File names cannot be read"
fileReadError = "Couldn't read files"


# Report messages:
saveToFileReport = "Data saved to the file: "

## Classifier Functions

In [ ]:
# Returns a list of samples files
# Helper Function
def nameFinder(folder):
    # filters only the .jpg files from the folder
    try:
        files = [f for f in listdir(folder) if isfile(join(folder, f))]
        imageNames = [f for f in files if f.split(".")[1] == "jpg"]
        return imageNames
    except:
        print (nameFinderError)

# Reads the sample files
def loadSamples(folder):
    sampleData = []
    # Loads files at ./tests folder to test based on the trained model 
    # only lists the .jpg files
    # collects all file names
    try:
        files = [f for f in listdir(folder) if isfile(join(folder, f))]
        # filters only the .jpg files
        images = [f for f in files if f.split(".")[1] == "jpg"]
        size = len(images)
        for img in images:
            newPath = folder+"/"+img
            newSample = tf.gfile.FastGFile(newPath, 'rb').read()
            sampleData.append(newSample)
        return sampleData
    except:
        print (fileReadError)

# Classifies the images
def classifier(sampleData, sampleNames):
    result = dict()
    size = len(sampleData)
    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("retrained_labels.txt")]
    # Unpersists graph from file
    with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        ####################################################
        # just a hacky way to solve version discrpancies
        # if using older versions of Tensorflow,
        # remove this line!
        del(graph_def.node[1].attr["dct_method"])
        ####################################################
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        fullResultLabels = []
        fullResultPred   = []
        # iterating over iamges
        for i in range (size):
        #for image_data_item in image_data:
            predictions = sess.run(softmax_tensor, \
                     {'DecodeJpeg/contents:0': sampleData[i]})
            # Sort to show labels of first prediction in order of confidence
            top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
            human_string = label_lines[top_k[0]]
            score = predictions[0][top_k[0]]

            # print report during the process
            if (printSwitch):
#                 print ("smaple ID", str(i))
#                 print('%s (score = %.2f)' % (human_string, score))
#                 print ("***********")
                tmpLabelList = []
                tmpPredList = []
                for j in range (len(label_lines)):
                    
                    tmpLabel =label_lines[top_k[j]] 
                    tmpPred = predictions[0][top_k[j]]
                    
                    tmpLabelList.append(tmpLabel)
                    tmpPredList.append(tmpPred)
                    
#                     print('%s (score = %.2f)' % (tmpLabel,tmpPred))
#                 print("------------------------------------")
                fullResultLabels.append(tmpLabelList)
                fullResultPred.append(tmpPredList)
                
            #if (human_string != "pass" or score < 0.5):
                #result[sampleNames[i]] = human_string
            result[sampleNames[i]] = human_string
        
    return (result,fullResultLabels,fullResultPred)

## Helper functions

In [ ]:
# this part must be edited heavily
# Prints a brief report at the end

def report(data):
    finish_time = time.time()
    ellapsed_time = finish_time - start_time
    average_time = ellapsed_time / float(len(data))

    # Print the final report
    print ('Total time:', str(int(ellapsed_time)))
    print ('Average time:', str(average_time)) 
    print("------------------------------------")
    print ("Results:")
    for key in data:
        print (key,"\t",data[key])
    print("------------------------------------")

# Writes data to file
# Helper function
def saveToFile(folder, logName,fails):
    # converts fails to json format
    jsonData = json.dumps(fails)

    # check if the log directory exist
    if not exists(folder):
        os.makedirs(folder)
    # generates the file name to save the log
    date_string = time.strftime("_%H:%M")
    newName = logName+date_string+".txt"
    newPath = os.path.join (folder, newName)

    # writes to file
    log = open(newPath, "w")
    log.write(str(jsonData))
    log.close()
    print (saveToFileReport,newPath)

# Copies failed cases to log folder with class added to the name
def saveFailSamples(logFolder,sampleFolder,fails):
    for fail in fails:
        failedSamplePath = sampleFolder+"/"+fail
        name= fail.split(".")
        targetPath = logFolder+"/"+name[0]+"_"+fails[fail]+".jpg"
        copyfile(failedSamplePath, targetPath)
    
# Generates the log in log folder 
def logger(logFolder,logFileName,fails,sampleFolder):
    saveToFile(logFolder, logFileName, fails)
    saveFailSamples(logFolder, sampleFolder, fails)

## Image preparation 

In [ ]:
def cropImage(images, t,b,l,r):
    croppedImages = []
    # find the cropped coordinations
    for i in range (len(images)):
        h , w,_  = images[i].shape
        xm = w//2
        ym = h//2
        x0 = xm - l
        x1 = xm + r
        y0 = ym - t
        y1 = ym + b
        # crop the image
        croppedImg = images[i][y0:y1, x0:x1]
        croppedImages.append(croppedImg)
    return croppedImages

def sliceImage(grid = 3, overlapVal = 0.5, path = None):
    cropCord = []
    keepAll = False
    if path == None:
        path = sampleFolder
    # convert the image to a grid for finding the object
    files = (listdir(sampleFolder))
    names = [f for f in files if f.split(".")[-1] == "jpg"]
    print (names)
    croppedImages = []
    # find the cropped coordinations
    for i in range (len(names)):
        name = names[i]
        tmpName = name.split(".")[0]
        name = os.path.join(path, name)
        image = cv2.imread(name)
        h , w,_  = image.shape
        width = w//grid
        overlap = width*overlapVal
        # crops the image in three sections, left, middle,right
        preStart = None
        preEnd = None
        for j in range (grid):
            start = j * width
            startOverlap = start - overlap//2
            if startOverlap < 0 : startOverlap = 0
            end = startOverlap + width + overlap//2
            if end > w : end = w
            if (preStart != startOverlap and preEnd != end) or keepAll:
                croppedImg = image[0:h, int(startOverlap):int(end)]
                cropCord.append((int(startOverlap),int(end)))
                tagName = os.path.join(path, tmpName+"_"+str(j)+".jpg")
                cv2.imwrite(tagName,croppedImg)
                croppedImages.append(croppedImg)
            preStart = startOverlap 
            preEnd = end 
        
    return croppedImages, cropCord

## Main body of codes

In [ ]:
def detection(slicing = False):
    
    # find original Images
    files = (listdir(sampleFolder))
    names = [f for f in files if f.split(".")[-1] == "jpg"]
    
    # slice Images
    if slicing :
        print ("slicing")
        slices,sliceCord = sliceImage(8, 3)
    samples = loadSamples(sampleFolder)
    
    # detect names and other stuff
    sampleNames = nameFinder(sampleFolder)
    print (sampleNames)
    print ("start classifying")
    
    results, labels, predicitons = classifier(samples,sampleNames)
    
    # Reporting
    logger(logFolder,logFileName,results,sampleFolder)
    report(results)
    print ("detection completed")
    return results, labels, predicitons,names, sliceCord

In [ ]:
def findMainObject(data):
    results, labels, predicitons,names, sliceCord = data
    probableLabel = []
    probablePred = []
    for i in range(len(labels)):
        probableLabel.append(labels[i][0])
        probablePred.append(labels[i][0])
    labels = set(probableLabel)
    labelData = dict()
    for label in labels:
        freq = 0
        for probLabel in probableLabel:
            if probLabel == label:
                freq+=1
        labelData[label] = freq
    print (labelData)
        
def HeadingFocusObject(data):
    results, labels, predicitons,names, sliceCord = data
    focusObject = labels[0][0]
    print ("Focus: ", focusObject)
    directionChart = []
    for i in range (1,len(labels)):
        prediction = predicitons[i][0]
        rate = 0
        if labels[i][0] == focusObject:
            if prediction > .5 : rate = 1 #i
        
        print (labels[i][0],prediction,rate )
        directionChart.append(rate)
    
    realDirection = np.mean(directionChart)
    print (realDirection) 
    return directionChart
    
def longestRun(data):
    print (data)
    LongestRun = []
    maxRunLen = 0
    curRun = []
    for i in range(len(data)):
        if data[i] == 1:
            curRun.append(i)
        else:
            curLen = len(curRun)
            if curLen > maxRunLen:
                LongestRun = curRun
            curRun = []
    curLen = len(curRun)
    if curLen > maxRunLen:
                LongestRun = curRun
    print (LongestRun)
    return LongestRun
            

In [ ]:
def findBearing(data):
    a,b,c,d,e = data
    fieldOfViewAngle = 90
    divisions = len(a)-1
    
    if divisions == 0:
        return None
    step = fieldOfViewAngle/divisions
    direction = HeadingFocusObject(data)
    longRun = longestRun(direction)
    bearing = np.mean(longRun)
    alpha = (bearing-3)*step
    print ("bearing: ", bearing)
    print ("divisions: ", divisions)
    print ("step: ", step)
    print ("alpha: ", alpha)
    return (alpha,longRun)

In [ ]:
#data = results, labels, predicitons
data = detection(True)
bearing, target = findBearing(data)
results, labels, predicitons,names, sliceCord = data

In [ ]:
#a,b,c,d,e = data
targetCor = []
for val in  (target):
    a,b = sliceCord[val]
    targetCor.extend([a,b])
targetCor.sort()
sourceName = names[0]
sourceName = os.path.join(sampleFolder, sourceName)
sourceImage = cv2.imread(sourceName)
print (type(sourceImage))
print (type(sourceImage))
# clean the folder
files = (listdir(sampleFolder))
names = [f for f in files if f.split(".")[-1] == "jpg"]
for name in names:
    name = os.path.join(sampleFolder, name)
    if name != sourceName:
        print(name)
        os.remove(name)
#cv2.imwrite(name,sourceImage) 

h = sourceImage.shape[0]
w = sourceImage.shape[1]
croppedImg = sourceImage[0:h,
                         targetCor[0]:targetCor[-1]]
plt.imshow(croppedImg)


In [ ]:
a,labels,c,d,e = data
print (labels[0][0])
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(sourceImage)

padding = 2
tPad = 20
wStart = targetCor[0]
hStart = 0
widthVal  = targetCor[-1]-targetCor[0]-padding
heightVal = h - padding

# Create a Rectangle patch
rect = patches.Rectangle((wStart,hStart),widthVal,heightVal,
                         linewidth=1,edgecolor='r',facecolor='none')
ax.text(wStart+tPad, 2*tPad,labels[0][0],color='r',fontsize=15)

# Add the patch to the Axes
ax.add_patch(rect)

plt.savefig(labels[0][0]+".jpg")
plt.show()
